In [1]:
import boto3

In [2]:
!pip3 install langchain-aws langchain-community langchain --quiet

## Llama3 70 Setting

In [3]:
from botocore.config import Config
from langchain_aws import ChatBedrock
bedrock_region = 'us-east-1'
modelId = "meta.llama3-70b-instruct-v1:0"
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    config=Config(
        retries = {
            'max_attempts': 30
        }            
    )
)

parameters = {
    "max_gen_len": 1024,  
    "top_p": 0.9, 
    "temperature": 0.1,
}    
chat = ChatBedrock(   
    model_id=modelId,
    client=boto3_bedrock, 
    model_kwargs=parameters,
)

In [4]:
from langchain.memory import ConversationBufferWindowMemory
memory_chain = ConversationBufferWindowMemory(memory_key="chat_history", output_key='answer', return_messages=True, k=10)

## General Conversation

In [5]:
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate

def general_conversation(chat, query):
    system = (
"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n
다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. 
AI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>"""
    )
    human = """<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    
    prompt = ChatPromptTemplate.from_messages([("system", system), MessagesPlaceholder(variable_name="history"), ("human", human)])
    print('prompt: ', prompt)
    
    chain = prompt | chat
        
    history = memory_chain.load_memory_variables({})["chat_history"]
    print('memory_chain: ', history)
                
    try: 
        stream = chain.invoke(
            {
                "history": history,
                "input": query,
            }
        )
        
        print('stream: ', stream)        
        usage = stream.response_metadata['usage']
        print('prompt_tokens: ', usage['prompt_tokens'])
        print('completion_tokens: ', usage['completion_tokens'])
        print('total_tokens: ', usage['total_tokens'])
        msg = stream.content
        
    except Exception:
        err_msg = traceback.format_exc()
        print('error message: ', err_msg)        
            
        raise Exception ("Not able to request to LLM")
    
    return msg

## 채팅 이력의 활용

In [6]:
text = "여행하고 싶어"
msg = general_conversation(chat, text)    
msg

prompt:  input_variables=['history', 'input'] input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. \nAI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'))]
memory_chain:  []
stream:  content='여행이란 좋은 취미를 가지고 계시군요! 요즘은 코로나19로 인해 여행이有点 제약이 있지만, 그래도 계획을 세워두면 언젠가 실현할 수 

'여행이란 좋은 취미를 가지고 계시군요! 요즘은 코로나19로 인해 여행이有点 제약이 있지만, 그래도 계획을 세워두면 언젠가 실현할 수 있을 겁니다. 어디로 여행을 가고 싶으신가요? 국내의某个 도시나 해외의 어떤 나라를 생각하고 계신가요?'

In [7]:
memory_chain.chat_memory.add_user_message(text)
memory_chain.chat_memory.add_ai_message(msg)

In [8]:
text = "제주도"
msg = general_conversation(chat, text)    
msg

prompt:  input_variables=['history', 'input'] input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. \nAI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'))]
memory_chain:  [HumanMessage(content='여행하고 싶어'), AIMessage(content='여행이란 좋은 취미를 가지고 계시군요! 요즘은 코로나19로 인해 여행이有点 

'제주도는 좋은 선택입니다! 제주도는 대한민국의 최남단에 위치해 있는 아름다운 섬으로, 해안절경과 다양한 자연경관이 있습니다. 특히 올레길, 성산일출봉, 만장굴 등은 제주도의 대표적인 관광지입니다.\n\n제주도에 가면 무엇을 하고 싶으신가요? 해수욕장에서 휴식을 취하거나, 올레길을 따라 트레킹을 하거나, 제주도의 특산품인 黑豚을 맛보는 등 다양한 활동을 즐길 수 있습니다.\n\n제주도에 가는 가장 좋은 시기는 4월부터 10월까지입니다. 이期间에는 날씨가 좋고, 다양한 축제와 행사가 열립니다. 특히 5월에는 제주해비치아트페스티벌, 7월에는 제주도국제장애인체육대회 등이 열립니다.\n\n제주도에 가는 방법은 여러 가지입니다. 가장 일반적인 방법은 제주국제공항에 도착하는 것입니다. 제주국제공항에는 인천국제공항, 김포국제공항, 부산김해국제공항 등에서 항공편이 있습니다. 또한, 제주도에는碼頭도 있어, 부산항, 목포항 등에서 페리를 타고 갈 수도 있습니다.'

In [9]:
memory_chain.chat_memory.add_user_message(text)
memory_chain.chat_memory.add_ai_message(msg)

In [10]:
memory_chain

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='여행하고 싶어'), AIMessage(content='여행이란 좋은 취미를 가지고 계시군요! 요즘은 코로나19로 인해 여행이有点 제약이 있지만, 그래도 계획을 세워두면 언젠가 실현할 수 있을 겁니다. 어디로 여행을 가고 싶으신가요? 국내의某个 도시나 해외의 어떤 나라를 생각하고 계신가요?'), HumanMessage(content='제주도'), AIMessage(content='제주도는 좋은 선택입니다! 제주도는 대한민국의 최남단에 위치해 있는 아름다운 섬으로, 해안절경과 다양한 자연경관이 있습니다. 특히 올레길, 성산일출봉, 만장굴 등은 제주도의 대표적인 관광지입니다.\n\n제주도에 가면 무엇을 하고 싶으신가요? 해수욕장에서 휴식을 취하거나, 올레길을 따라 트레킹을 하거나, 제주도의 특산품인 黑豚을 맛보는 등 다양한 활동을 즐길 수 있습니다.\n\n제주도에 가는 가장 좋은 시기는 4월부터 10월까지입니다. 이期间에는 날씨가 좋고, 다양한 축제와 행사가 열립니다. 특히 5월에는 제주해비치아트페스티벌, 7월에는 제주도국제장애인체육대회 등이 열립니다.\n\n제주도에 가는 방법은 여러 가지입니다. 가장 일반적인 방법은 제주국제공항에 도착하는 것입니다. 제주국제공항에는 인천국제공항, 김포국제공항, 부산김해국제공항 등에서 항공편이 있습니다. 또한, 제주도에는碼頭도 있어, 부산항, 목포항 등에서 페리를 타고 갈 수도 있습니다.')]), output_key='answer', return_messages=True, memory_key='chat_history', k=10)

In [11]:
!pip3 install bs4 --quiet

In [12]:
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool

@tool
def get_product_list(keyword: str) -> list:
    """
    Search product list by keyword and then return product list
    keyword: search keyword
    return: product list
    """

    url = f"https://search.kyobobook.co.kr/search?keyword={keyword}&gbCode=TOT&target=total"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        prod_info = soup.find_all("a", attrs={"class": "prod_info"})
        prod_list = [
            {"title": prod.text.strip(), "link": prod.get("href")} for prod in prod_info
        ]
        return prod_list[:5]
    else:
        return []

In [13]:
prod_list = get_product_list("여행")
prod_list

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[{'title': '[국내도서]\n예약판매\n스마트폰과 함께하는 디지털여행',
  'link': 'https://product.kyobobook.co.kr/detail/S000213314396'},
 {'title': '[국내도서]\n예약판매\n인생은 여행',
  'link': 'https://product.kyobobook.co.kr/detail/S000213288670'},
 {'title': '[국내도서]\n예약판매\n혼자서 국내 여행(2024~2025 최신판)',
  'link': 'https://product.kyobobook.co.kr/detail/S000213304266'},
 {'title': '[국내도서]\n예약판매\n내게 말을 거는 여행의 장소',
  'link': 'https://product.kyobobook.co.kr/detail/S000213290232'},
 {'title': '[국내도서]\n여행의 이유',
  'link': 'https://product.kyobobook.co.kr/detail/S000212972861'}]

In [14]:
# langchain agent tools
tools = [get_product_list]

In [15]:
#chat = chat.bind(function=tools)
chat = chat.bind_tools(tools)

In [16]:
query = '안녕'

In [17]:
import traceback

In [23]:
system = (
"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n
다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. 
AI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>"""
    )
human = """<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    
# prompt = ChatPromptTemplate.from_messages([("system", system), MessagesPlaceholder(variable_name="agent_scratchpad"), ("human", human)])

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder(variable_name="history"),
    ("user", human),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

chain = prompt | chat
        
history = memory_chain.load_memory_variables({})["chat_history"]
print('memory_chain: ', history)
                
try: 
    stream = chain.invoke(
        {
            "history": history,
            "input": query,
            "agent_scratchpad": []
        }
    )
        
    print('stream: ', stream)        
    usage = stream.response_metadata['usage']
    print('prompt_tokens: ', usage['prompt_tokens'])
    print('completion_tokens: ', usage['completion_tokens'])
    print('total_tokens: ', usage['total_tokens'])
    msg = stream.content
except Exception:
    err_msg = traceback.format_exc()
    print('error message: ', err_msg)        
            
    raise Exception ("Not able to request to LLM")

memory_chain:  [HumanMessage(content='여행하고 싶어'), AIMessage(content='여행이란 좋은 취미를 가지고 계시군요! 요즘은 코로나19로 인해 여행이有点 제약이 있지만, 그래도 계획을 세워두면 언젠가 실현할 수 있을 겁니다. 어디로 여행을 가고 싶으신가요? 국내의某个 도시나 해외의 어떤 나라를 생각하고 계신가요?'), HumanMessage(content='제주도'), AIMessage(content='제주도는 좋은 선택입니다! 제주도는 대한민국의 최남단에 위치해 있는 아름다운 섬으로, 해안절경과 다양한 자연경관이 있습니다. 특히 올레길, 성산일출봉, 만장굴 등은 제주도의 대표적인 관광지입니다.\n\n제주도에 가면 무엇을 하고 싶으신가요? 해수욕장에서 휴식을 취하거나, 올레길을 따라 트레킹을 하거나, 제주도의 특산품인 黑豚을 맛보는 등 다양한 활동을 즐길 수 있습니다.\n\n제주도에 가는 가장 좋은 시기는 4월부터 10월까지입니다. 이期间에는 날씨가 좋고, 다양한 축제와 행사가 열립니다. 특히 5월에는 제주해비치아트페스티벌, 7월에는 제주도국제장애인체육대회 등이 열립니다.\n\n제주도에 가는 방법은 여러 가지입니다. 가장 일반적인 방법은 제주국제공항에 도착하는 것입니다. 제주국제공항에는 인천국제공항, 김포국제공항, 부산김해국제공항 등에서 항공편이 있습니다. 또한, 제주도에는碼頭도 있어, 부산항, 목포항 등에서 페리를 타고 갈 수도 있습니다.')]
stream:  content='안녕하세요, 서연입니다. 방금 제주도 여행에 대한 이야기를 나누었습니다. 혹시 다른 여행 계획이나 질문이 있으신가요?' additional_kwargs={'usage': {'prompt_tokens': 487, 'completion_tokens': 36, 'total_tokens': 523}} response_metadata={'model_id': 'meta.llama3-70b-instruct-v1:0', 'usage': {'prompt_tok

In [24]:
msg

'안녕하세요, 서연입니다. 방금 제주도 여행에 대한 이야기를 나누었습니다. 혹시 다른 여행 계획이나 질문이 있으신가요?'

In [25]:
system

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. \nAI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>'

In [26]:
human

'<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [27]:
query

'안녕'

In [31]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder(variable_name="history"),
    ("human", human),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [32]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(chat,tools,prompt)

ValueError: Prompt missing required variables: {'tool_names', 'tools'}

In [30]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    return_intermediate_steps=True
)

NameError: name 'agent' is not defined

In [ ]:
response = agent_executor.invoke(
    {
        "input": query,
        "agent_scratchpad": []
    }
)

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
agent = create_react_agent(chat,tools,prompt)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    return_intermediate_steps=True
)

In [ ]:
response = agent_executor.invoke({"input": "안녕, 반가워!"})
print(f'답변: {response["output"]}')

In [ ]:
print(response)

In [ ]:
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
)

In [ ]:
def run(agent_executor: AgentExecutor, keyword: str) -> dict:
    result = agent_executor.invoke(
        {
            "input": f"""
        1. `{keyword}` 키워드로 상품 리스트를 가져와줘.
        2. 1번 과정에서 가져온 상품 리스트 각각의 table of content 를 가져와줘.
        3. 마지막으로 모든 상품 리스트의 table of content 를 종합하여 내가 앞으로 집필할 베스트셀러 책의 목차를 작성해 줘.
        """
        }
    )
    return result["output"]

In [ ]:
final_answer = run(agent_executor, "Pandas")